In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv("/kaggle/input/gm-chess-games/chess/games_chesstempo.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158454 entries, 0 to 158453
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id_game          158454 non-null  object 
 1   moves            158454 non-null  object 
 2   date             154146 non-null  object 
 3   white_player     154146 non-null  object 
 4   id_white_player  154146 non-null  object 
 5   black_player     154146 non-null  object 
 6   id_black_player  154146 non-null  object 
 7   white_elo        154146 non-null  float64
 8   black_elo        154146 non-null  float64
 9   avg_elo          154146 non-null  float64
 10  event            154146 non-null  object 
 11  site             154146 non-null  object 
 12  eco              154146 non-null  object 
 13  opening          154146 non-null  object 
 14  id_opening       154146 non-null  object 
 15  round            154146 non-null  object 
 16  result           154146 non-null  obje

In [ ]:
import chess.pgn
import io

def pgnToFenList(moves):
    pgn = io.StringIO(moves)
    game = chess.pgn.read_game(pgn)
    board = game.board()
    fenList = []
    for move in game.mainline_moves():
        board.push(move)
        fen = board.fen()
        fenList.append(fen)
    return fenList

In [ ]:
def position(fen):
    fen_parts = fen.split(' ')
    position = fen_parts[0]
    return position

In [9]:
# df['fen'] = df['moves'].apply(pgnToFenList)

In [2]:
# Explode the "fen" column
#df = df.explode("fen")
# Reset the index
#df.reset_index(drop=True, inplace=True)

#df['position'] = df['fen'].apply(position)
#df["position"]

In [4]:
df = df[["id_game", "moves", "n_moves", "white_elo", "black_elo", "result"]]
df.head()

,id_game,moves,n_moves,white_elo,black_elo,result
0,https://old.chesstempo.com/gamedb/game/4820223,1.e4 e5 2.Nf3 Nc6 3.Bc4 Bc5 4.c3 Nf6 5.d4 exd4...,36.0,2665.0,2598.0,1-0
1,https://old.chesstempo.com/gamedb/game/4819358,1.e4 c5 2.Nf3 d6 3.d4 cxd4 4.Nxd4 Nf6 5.Nc3 a6...,21.0,2665.0,2449.0,1-0
2,https://old.chesstempo.com/gamedb/game/4818704,1.d4 Nf6 2.c4 g6 3.Nc3 Bg7 4.e4 d6 5.Be2 O-O 6...,37.0,2665.0,2535.0,0-1
3,https://old.chesstempo.com/gamedb/game/4818558,1.d4 Nf6 2.c4 e6 3.g3 d5 4.Bg2 Be7 5.Nf3 O-O 6...,32.0,2528.0,2665.0,½-½
4,https://old.chesstempo.com/gamedb/game/4820371,1.e4 e5 2.Nf3 Nf6 3.Nxe5 d6 4.Nf3 Nxe4 5.c4 Nc...,26.0,2665.0,2331.0,1-0


In [ ]:
df["avg_elo"] = (df["white_elo"] + df["black_elo"])/2
df["avg_elo"]

In [ ]:
df["avg_elo"].mean()

In [ ]:
df["avg_elo"].std()

In [ ]:
# Hypothesis 1. Maia chess engine (1900,...) is good to represent humans in statistical sample for experimentation.
# Hypothesis 2. Maia chess engine, feeded by custom data (my data), can not solve chess puzzles as efficient as humans with same training data.

In [50]:
# I is an index for parties by hour that can be played to absorb infromation for all the positions
# every position in average has N bits of information - Nbits
# every party has 85 positions in average
# Ngames are played games in an hour
# how many seconds does GM needs to process a position or a game / how many information can player absorb in an hour
# number of minutes in GM games
# in average players have a game of X minutes in total
# How many minutes per position

sec_positionIconic = 0.3
sec_positionRecognition = 0.5 # probably a range due to level of correctnessć
sec_chunkRecog = 1
sec_learnChunk = 8
sec_chunkExpand = 2
sec_positionRecognition = sec_chunkRecog * 3

nPositions = df["n_moves"].mean()*2
depth = 10
options = 10 # options to consider
iiconic = sec_positionIconic*nPositions/60 # minutes
irecognition = sec_positionRecognition*nPositions*depth*options/60 # minutes

ilearningNew = sec_learnChunk*3*nPositions*depth*options/60 # minutes
ilearningExpend = sec_chunkExpand*3*nPositions*depth*options/60 # minutes

In [51]:
print(f"time to process {nPositions} positions in iconic memory is -----------------------{iiconic} minutes")
print(f"time to process {nPositions} positions in terms of recognition in STM is ---------{irecognition} minutes")
print(f"time to process {nPositions} positions in terms of learning new chunk in STM is --{ilearningNew} minutes")
print(f"time to process {nPositions} positions in terms of expanding know chunk in STM is {ilearningExpend} minutes")

time to process 86.34045645037821 positions in iconic memory is -----------------------0.43170228225189106 minutes
time to process 86.34045645037821 positions in terms of recognition in STM is ---------431.70228225189106 minutes
time to process 86.34045645037821 positions in terms of learning new chunk in STM is --3453.6182580151285 minutes
time to process 86.34045645037821 positions in terms of expanding know chunk in STM is 863.4045645037821 minutes


In [57]:
200*60*14600*0.7/86

1426046.5116279067

In [54]:
totalExHours = 4*365*10 # 4 hours in 10 years 
totalExGames = totalExHours * 12 # number of hours * number of played games in hour
totalExPositions = totalExGames * 86

print(f"Chess expert in his {totalExHours} hours total in 10 years of playing every day for 8 hours, played {totalExGames} games and saw {round(totalExPositions)} positions")
# 20 hours/week * 52 weeks/year * 11 years = 12,480 hours
# https://www.chess.com/article/view/chess-grandmaster-hours

Chess expert in his 14600 hours total in 10 years of playing every day for 8 hours, played 175200 games and saw 15067200 positions


In [93]:
min_iconicMemorySec = 0.15
min_visualSTMsec = 5
sec_pos = (min_iconicMemorySec+min_visualSTMsec)/2
pos_min = 60/sec_pos
pos_hour = 60/pos_min
positionsIn10years = pos_hour*60*14600  # 2255700.0 positions
gamesIn10years = positionsIn10years/86  # 26229 games

In [95]:
pos_hour

2.575

In [83]:
4*365*10

14600

In [96]:
pos_hour

2.575